In [ ]:
import numpy as np
import nibabel as nib
import glob
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import random
import keras
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()

In [ ]:
#change the path
data_path= '/content/drive/My Drive/Thesis_implementation_files/DATASET_copy(20)/DATASET_BRATS2020/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'

In [ ]:
#making list by collecting same type images and bringing them together
T1_list = sorted(glob.glob( data_path+ '/*/*t1.nii'))
T2_list = sorted(glob.glob(data_path + '/*/*t2.nii'))
T1ce_list = sorted(glob.glob(data_path + '/*/*t1ce.nii'))
flair_list = sorted(glob.glob(data_path + '/*/*flair.nii'))
mask_list = sorted(glob.glob(data_path + '/*/*seg.nii'))

In [ ]:
test_mask= nib.load(data_path + 'BraTS20_Training_049/BraTS20_Training_049_seg.nii').get_fdata()
test_mask= test_mask.astype(np.uint8)
print(np.unique(test_mask))

test_mask[test_mask == 4]= 3
print(np.unique(test_mask))


In [ ]:
#running a loop through the lists considering all the list has same length as T1
#then sorting them out if the file is to be saved or discarded 
#Boundary Box:
import scipy.ndimage.measurements as mnts
list_length = len(T1_list)
image_num = 0

for img in range(list_length):  
         print("Now preparing image and masks number: ", img)

         temp_image_T1=nib.load(T1_list[img]).get_fdata()
  
         temp_image_T2=nib.load(T2_list[img]).get_fdata()

         temp_image_T1ce=nib.load(T1ce_list[img]).get_fdata()
        
         temp_mask=nib.load(mask_list[img]).get_fdata()
         temp_mask=temp_mask.astype(np.uint8)
         temp_mask[temp_mask==4] = 3  
      
            
         finalmaxX = 0
         finalmaxY = 0
         finalminX = 0
         finalminY = 0
         for n_slice in range(0, temp_image_T1.shape[2]):
             img = temp_image_T1[:,:,n_slice]
             image = np.where(img > 0, 1, 0)

             structure = np.array([
                       [1,1,1],
                       [1,1,1],
                       [1,1,1]
                      ])

             bboxSlices = {}
             for i in range(1, image.max() + 1):
                 B = image.copy()
                 B[B != i] = 0

                 bboxSlices[i] = mnts.find_objects(mnts.label(B, structure=structure)[0])
             if(len(bboxSlices)>0):
                 bboxes = []
                 for bboxslices in bboxSlices[1]:
                      bbox = []
                      for slice in bboxslices:
                          bbox.append(slice.start)
                          bbox.append(slice.stop)
                      bboxes.append(bbox)


                 bboxes_np = np.array(bboxes, dtype=np.float32)

                 ymin = np.min(bboxes_np[:,0])
                 ymax = np.max(bboxes_np[:,1])
                 xmin = np.min(bboxes_np[:,2])
                 xmax = np.max(bboxes_np[:,3])
                 
                 if ((abs(ymax - ymin) * abs(xmax - xmin)) > (abs(finalmaxY - finalminY) * abs(finalmaxX - finalminY))):
                    finalmaxY = ymax
                    finalminY = ymin
                    finalmaxX = xmax
                    finalminX = xmin

   
         img = temp_image_T1[:,:,0]
         combined_images_T1 = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         combined_images_T2 = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         combined_images_T1ce = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         combined_mask = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         
                                  
         finalT1 = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         finalT2 = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         finalT1ce = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
         finalmask = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]

                                    
         for n_slice in range(1, temp_image_T1.shape[2]):
            img = temp_image_T1[:,:,n_slice]
            finalT1 = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]

            img = temp_image_T2[:,:,n_slice]
            finalT2 = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]

            img = temp_image_T1ce[:,:,n_slice]
            finalT1ce = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]
          
            img = temp_mask[:,:,n_slice]
            finalmask = img[int(finalminY):int(finalmaxY), int(finalminX):int(finalmaxX)]


            combined_images_T1 = np.dstack((combined_images_T1, finalT1))
            combined_images_T2 = np.dstack((combined_images_T2, finalT2))
            combined_images_T1ce = np.dstack((combined_images_T1ce, finalT1ce))
            combined_mask = np.dstack((combined_mask, finalmask))
         combined_images_T1 = resize(combined_images_T1, (128, 128, 128))
         combined_images_T2 = resize(combined_images_T2, (128, 128, 128))
         combined_images_T1ce = resize(combined_images_T1ce, (128, 128, 128))
         combined_mask = resize(combined_mask, (128, 128, 128))

         #plotting and showing the image.
         #n_slice= random.randint(0, test_mask.shape[2])
         n_slice= 116
         plt.figure(figsize= (12, 8))

         plt.subplot(231)
         plt.imshow(combined_images_T1[:,:,n_slice], cmap='gray')
         plt.title('Image T1')

         plt.subplot(232)
         plt.imshow(combined_images_T1ce[:,:,n_slice], cmap='gray')
         plt.title('Image T1ce')

         plt.subplot(233)
         plt.imshow(combined_images_T2[:,:,n_slice], cmap='gray')
         plt.title('Image T2')

         plt.subplot(234)
         plt.imshow(combined_mask[:,:,n_slice])
         plt.title('Mask')

         plt.show()

         print("T1=", combined_images_T1.shape)
         print("T2=", combined_images_T2.shape)
         print("T1ce=", combined_images_T1ce.shape)

         
         final_image = np.stack((combined_images_T1, combined_images_T2, combined_images_T1ce), axis = -1)
         print("Final image shape=", final_image.shape)
            
         print("Final mask shape=", combined_mask.shape)
    
         val, counts = np.unique(temp_mask, return_counts=True)
         
         if((1 - (counts[0]/counts.sum())) > 0.01):  
            print("Save Me")
            temp_mask= to_categorical(temp_mask, num_classes=4)
            my_path=  '/content/drive/My Drive/Thesis_implementation_files/DATASET_copy(20)/DATASET_BRATS2020/BraTS2020_TrainingData'
            np.save(my_path + '/input_data_3channels/images/image_' + str(image_num) + '.npy', final_image)
            np.save(my_path + '/input_data_3channels/masks/mask_' + str(image_num) + '.npy', temp_mask)
         else:
            print("Discard me") 
            
         image_num += 1
 

In [ ]:
import splitfolders

In [ ]:
input_folder = 'BraTS2020_TrainingData/input_data_3channels/'
output_folder = 'BraTS2020_TrainingData/input_data_128/'

In [ ]:
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.75, .25), group_prefix=None)